**TRABAJO PRACTICO N°1: PROCESAMIENTO DEL LENGUAJE NATURAL**

**LIBRERIAS**

In [1]:
!pip install sentence-transformers
!python -m spacy download es_core_news_md
!pip install nltk
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 21.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.4 MB/s eta 0:00:00


In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from nltk.corpus import stopwords
import nltk
import spacy

Hacemos extracción, de la página Gutenberg, los libros más descargados con sus respectivos autores. Realizando un CSV con dicha información (título y autor).

In [3]:
url = "https://www.gutenberg.org/browse/scores/top1000.php#books-last1"

# Realizar la solicitud a la página web
response = requests.get(url)

# Parsear el contenido HTML
soup = BeautifulSoup(response.content, 'html.parser')

book_elements = soup.find_all('li')

# Crear una lista para almacenar los títulos y autores de los libros
books = []

# Recorrer cada elemento y extraer el título y el autor
for element in book_elements:
    text = element.get_text()
    if 'by' in text:
        parts = text.split('by')
        title = parts[0].strip()  # El título del libro
        author = parts[1].strip()  # El autor del libro
        books.append({'Title': title, 'Author': author})


books_df = pd.DataFrame(books)


print(books_df)

books_df.to_csv('gutenberg_books.csv', index=False)



                                        Title  \
0     Frankenstein; Or, The Modern Prometheus   
1                         Pride and Prejudice   
2                          The Scarlet Letter   
3                                          Mo   
4                            Romeo and Juliet   
...                                       ...   
2825        A handy dictionary of mythology :   
2826      The little book of life after death   
2827                                     Orms   
2828                                      Der   
2829                                      Sel   

                                         Author  
0            Mary Wollstonecraft Shelley (8508)  
1                            Jane Austen (2920)  
2                    Nathaniel Hawthorne (2562)  
3                           Dick; Or, The Whale  
4                    William Shakespeare (2426)  
...                                         ...  
2825                      Thomas Preston (1223)  
2826       

Abrimos tres archivos CSV que tienen información sobre juegos de mesa, películas y libros.

In [4]:
board_games_df = pd.read_csv('bgg_database.csv')

movies_df = pd.read_csv('IMDB-Movie-Data.csv')

books_df = pd.read_csv('gutenberg_books.csv')

#Imprimimos las primeras filas de cada CSV
print(board_games_df.head())
print(movies_df.head())
print(books_df.head())


   rank                          game_name  \
0     1                  Brass: Birmingham   
1     2          Pandemic Legacy: Season 1   
2     3                         Gloomhaven   
3     4                           Ark Nova   
4     5  Twilight Imperium: Fourth Edition   

                                           game_href  geek_rating  avg_rating  \
0  https://boardgamegeek.com/boardgame/224517/bra...        8.415        8.60   
1  https://boardgamegeek.com/boardgame/161936/pan...        8.377        8.53   
2  https://boardgamegeek.com/boardgame/174430/glo...        8.349        8.59   
3  https://boardgamegeek.com/boardgame/342942/ark...        8.335        8.54   
4  https://boardgamegeek.com/boardgame/233078/twi...        8.240        8.60   

   num_voters                                        description  \
0     46836.0  Brass: Birmingham is an economic strategy game...   
1     53807.0  Pandemic Legacy is a co-operative campaign gam...   
2     62592.0  Gloomhaven  is a 

**CLASIFICADOR CON REGRESION LOGISTICA**

En este caso, utilizamos TF-IDF Vectorizer.
Usando dicho modelo de regresión logística para clasificar frases en Alegre, Melancólico o ni fu ni fa.



In [23]:
nltk.download('stopwords')

spanish_stop_words = stopwords.words('spanish')

# Etiquetas de estados de ánimo
labels = [(0, "Alegre"), (1, "Melancólico"), (2, "Ni fu ni fa")]


dataset = [
    # Frases de estado de ánimo "Alegre"
    (0, "Estoy muy feliz"),
    (0, "Hoy es un gran día"),
    (0, "Me siento alegre"),
    (0, "Es un día increíble"),
    (0, "Estoy lleno de energía"),
    (0, "Estoy muy contento hoy"),
    (0, "Este es un día perfecto"),
    (0, "Me siento lleno de vida"),
    (0, "Estoy eufórico"),
    (0, "Hoy es un día para celebrar"),
    (0, "Siento mucha alegría en este día"),
    (0, "Todo es perfecto hoy"),
    (0, "Me encanta la vida"),
    (0, "Es un día espectacular para disfrutar"),
    (0, "Hoy estoy muy entusiasmado"),
    (0, "Me siento agradecido y optimista"),
    (0, "Qué día tan brillante y hermoso"),
    (0, "Estoy exultante de felicidad"),
    (0, "Todo está saliendo a la perfección"),
    (0, "Estoy lleno de emociones positivas"),
    (0, "Siento una gran paz y alegría"),
    (0, "Hoy es un día lleno de posibilidades"),
    (0, "Nada puede arruinar mi buen humor"),
    (0, "Todo se siente más fácil y agradable"),
    (0, "Es un día maravilloso para vivir"),

    # Frases de estado de ánimo "Melancólico"
    (1, "Me siento triste"),
    (1, "Estoy melancólico"),
    (1, "Hoy estoy nostálgico"),
    (1, "Me siento deprimido"),
    (1, "No puedo dejar de pensar en cosas tristes"),
    (1, "Es un día gris"),
    (1, "Me siento vacío"),
    (1, "No tengo ánimos para nada"),
    (1, "Estoy abatido"),
    (1, "Todo me parece sombrío"),
    (1, "Siento una gran tristeza"),
    (1, "No me gusta este día"),
    (1, "Es un día muy malo"),
    (1, "Todo parece estar mal hoy"),
    (1, "Nada me da consuelo hoy"),
    (1, "Siento que todo es una carga"),
    (1, "Estoy atrapado en mis pensamientos"),
    (1, "No tengo energías para nada"),
    (1, "Es un día sin color ni esperanza"),
    (1, "Nada me hace sonreír hoy"),
    (1, "No encuentro alegría en nada"),
    (1, "Es un día de pura tristeza y soledad"),
    (1, "Siento un vacío muy profundo"),
    (1, "Nada tiene sentido hoy"),
    (1, "Me siento como una sombra"),

    # Frases de estado de ánimo "Ni fu ni fa"
    (2, "No tengo ganas de hacer nada"),
    (2, "Me siento neutral"),
    (2, "Estoy cansado"),
    (2, "Me da igual lo que pase"),
    (2, "No me importa nada hoy"),
    (2, "Estoy indiferente"),
    (2, "No siento ni alegría ni tristeza"),
    (2, "Hoy es un día normal"),
    (2, "Estoy aburrido"),
    (2, "Nada me parece interesante"),
    (2, "No sé cómo me siento"),
    (2, "Todo está igual que siempre"),
    (2, "Nada especial sucede hoy"),
    (2, "Es un día cualquiera"),
    (2, "Hoy me siento indiferente a todo"),
    (2, "Simplemente no siento nada"),
    (2, "Es uno de esos días sin nada nuevo"),
    (2, "Ni me siento bien ni mal"),
    (2, "Nada me motiva particularmente"),
    (2, "Es un día más, sin nada que contar"),
    (2, "No tengo emociones intensas hoy"),
    (2, "Hoy no pasa nada fuera de lo común"),
    (2, "Es un día bastante monótono"),
    (2, "Todo está en el mismo estado de siempre"),
    (2, "Ni estoy triste ni contento, simplemente estoy")
]

# Preparar X (frases) e y (etiquetas de estados de ánimo)
X = [text.lower() for label, text in dataset]
y = [label for label, text in dataset]

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorización del texto con TF-IDF, eliminando palabras vacías y usando bigramas
vectorizer = TfidfVectorizer(stop_words=spanish_stop_words, ngram_range=(1, 2))
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Crear y entrenar el modelo de Regresión Logística con balance de clases
modelo_LR = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs', class_weight='balanced')
modelo_LR.fit(X_train_vectorized, y_train)

# Evaluar el modelo con el conjunto de prueba
y_pred_LR = modelo_LR.predict(X_test_vectorized)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

print("Precisión Regresión Logística:", acc_LR)
print("Reporte de clasificación Regresión Logística:\n", report_LR)



Precisión Regresión Logística: 0.3333333333333333
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

           0       0.50      0.50      0.50         6
           1       0.00      0.00      0.00         5
           2       0.25      0.50      0.33         4

    accuracy                           0.33        15
   macro avg       0.25      0.33      0.28        15
weighted avg       0.27      0.33      0.29        15



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Contamos con una precisión del 44% siendo el porcentaje de clasificaciones correctas.

Probamos el modelo de clasificación:

In [24]:
def predict_mood(user_input):
    user_input_vectorized = vectorizer.transform([user_input.lower()])
    mood_label = modelo_LR.predict(user_input_vectorized)[0]
    return labels[mood_label][1]


frase_usuario = "Hoy me siento muy triste"
estado_animo = predict_mood(frase_usuario)
print(f"El estado de ánimo detectado es: {estado_animo}")

# Nuevas frases más distintas para probar el modelo
nuevas_frases = [
    "No puedo evitar sentir una gran tristeza hoy",  # Debería ser "Melancólico"
    "Siento que este es uno de los mejores días de mi vida",  # Debería ser "Alegre"
    "Hoy todo me da lo mismo, nada me interesa realmente",  # Debería ser "Ni fu ni fa"
    "No sé qué hacer, pero no estoy ni triste ni feliz",  # Debería ser "Ni fu ni fa"
    "Me siento increíblemente emocionado y lleno de energía",  # Debería ser "Alegre"
    "Todo parece gris y no veo nada que me anime",  # Debería ser "Melancólico"
    "Este día está siendo tan normal como cualquier otro",  # Debería ser "Ni fu ni fa"
    "No hay nada que me entusiasme, es como si estuviera vacío",  # Debería ser "Melancólico"
    "Estoy muy agradecido por todo, este día es fantástico",  # Debería ser "Alegre"
    "Hoy no me pasa nada en particular, no siento ni tristeza ni alegría"  # Debería ser "Ni fu ni fa"
]

# Predecir los estados de ánimo para las nuevas frases
for frase in nuevas_frases:
    print(f"La frase '{frase}' tiene el estado de ánimo: {predict_mood(frase)}")


El estado de ánimo detectado es: Melancólico
La frase 'No puedo evitar sentir una gran tristeza hoy' tiene el estado de ánimo: Melancólico
La frase 'Siento que este es uno de los mejores días de mi vida' tiene el estado de ánimo: Ni fu ni fa
La frase 'Hoy todo me da lo mismo, nada me interesa realmente' tiene el estado de ánimo: Ni fu ni fa
La frase 'No sé qué hacer, pero no estoy ni triste ni feliz' tiene el estado de ánimo: Ni fu ni fa
La frase 'Me siento increíblemente emocionado y lleno de energía' tiene el estado de ánimo: Alegre
La frase 'Todo parece gris y no veo nada que me anime' tiene el estado de ánimo: Melancólico
La frase 'Este día está siendo tan normal como cualquier otro' tiene el estado de ánimo: Alegre
La frase 'No hay nada que me entusiasme, es como si estuviera vacío' tiene el estado de ánimo: Melancólico
La frase 'Estoy muy agradecido por todo, este día es fantástico' tiene el estado de ánimo: Alegre
La frase 'Hoy no me pasa nada en particular, no siento ni tristez

**EN ESTE CASO, UTILIZAMOS SENTENCE TRANSFORMER**

Entrenamos un modelo de clasificación (de estado de ánimo) pero en este caso, utilizando embeddings generados por el modelo de SentenceTransforme.

In [29]:
#model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
model = SentenceTransformer('intfloat/multilingual-e5-small')

labels = [(0, "Alegre"), (1, "Melancólico"), (2, "Ni fu ni fa")]


dataset = [
    # Frases de estado de ánimo "Alegre"
    (0, "Estoy muy feliz"),
    (0, "Hoy es un gran día"),
    (0, "Me siento alegre"),
    (0, "Es un día increíble"),
    (0, "Estoy lleno de energía"),
    (0, "Estoy muy contento hoy"),
    (0, "Este es un día perfecto"),
    (0, "Me siento lleno de vida"),
    (0, "Estoy eufórico"),
    (0, "Hoy es un día para celebrar"),
    (0, "Siento mucha alegría en este día"),
    (0, "Todo es perfecto hoy"),
    (0, "Me encanta la vida"),
    (0, "Es un día espectacular para disfrutar"),
    (0, "Hoy estoy muy entusiasmado"),
    (0, "Me siento agradecido y optimista"),
    (0, "Qué día tan brillante y hermoso"),
    (0, "Estoy exultante de felicidad"),
    (0, "Todo está saliendo a la perfección"),
    (0, "Estoy lleno de emociones positivas"),
    (0, "Siento una gran paz y alegría"),
    (0, "Hoy es un día lleno de posibilidades"),
    (0, "Nada puede arruinar mi buen humor"),
    (0, "Todo se siente más fácil y agradable"),
    (0, "Es un día maravilloso para vivir"),

    # Frases de estado de ánimo "Melancólico"
    (1, "Me siento triste"),
    (1, "Estoy melancólico"),
    (1, "Hoy estoy nostálgico"),
    (1, "Me siento deprimido"),
    (1, "No puedo dejar de pensar en cosas tristes"),
    (1, "Es un día gris"),
    (1, "Me siento vacío"),
    (1, "No tengo ánimos para nada"),
    (1, "Estoy abatido"),
    (1, "Todo me parece sombrío"),
    (1, "Siento una gran tristeza"),
    (1, "No me gusta este día"),
    (1, "Es un día muy malo"),
    (1, "Todo parece estar mal hoy"),
    (1, "Nada me da consuelo hoy"),
    (1, "Siento que todo es una carga"),
    (1, "Estoy atrapado en mis pensamientos"),
    (1, "No tengo energías para nada"),
    (1, "Es un día sin color ni esperanza"),
    (1, "Nada me hace sonreír hoy"),
    (1, "No encuentro alegría en nada"),
    (1, "Es un día de pura tristeza y soledad"),
    (1, "Siento un vacío muy profundo"),
    (1, "Nada tiene sentido hoy"),
    (1, "Me siento como una sombra"),

    # Frases de estado de ánimo "Ni fu ni fa"
    (2, "No tengo ganas de hacer nada"),
    (2, "Me siento neutral"),
    (2, "Estoy cansado"),
    (2, "Me da igual lo que pase"),
    (2, "No me importa nada hoy"),
    (2, "Estoy indiferente"),
    (2, "No siento ni alegría ni tristeza"),
    (2, "Hoy es un día normal"),
    (2, "Estoy aburrido"),
    (2, "Nada me parece interesante"),
    (2, "No sé cómo me siento"),
    (2, "Todo está igual que siempre"),
    (2, "Nada especial sucede hoy"),
    (2, "Es un día cualquiera"),
    (2, "Hoy me siento indiferente a todo"),
    (2, "Simplemente no siento nada"),
    (2, "Es uno de esos días sin nada nuevo"),
    (2, "Ni me siento bien ni mal"),
    (2, "Nada me motiva particularmente"),
    (2, "Es un día más, sin nada que contar"),
    (2, "No tengo emociones intensas hoy"),
    (2, "Hoy no pasa nada fuera de lo común"),
    (2, "Es un día bastante monótono"),
    (2, "Todo está en el mismo estado de siempre"),
    (2, "Ni estoy triste ni contento, simplemente estoy")
]

# Preparar X (frases) e y (etiquetas de estados de ánimo)
X = [text.lower() for label, text in dataset]
y = [label for label, text in dataset]

# Generar los embeddings para las frases usando el modelo preentrenado
X_embeddings = model.encode(X)

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_embeddings, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de Regresión Logística
modelo_LR = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs', class_weight='balanced')
modelo_LR.fit(X_train, y_train)

# Evaluación del modelo
y_pred_LR = modelo_LR.predict(X_test)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)
print("Precisión Regresión Logística:", acc_LR)
print("Reporte de clasificación Regresión Logística:\n", report_LR)

Precisión Regresión Logística: 0.8
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       0.80      0.80      0.80         5
           2       0.67      0.50      0.57         4

    accuracy                           0.80        15
   macro avg       0.77      0.77      0.76        15
weighted avg       0.79      0.80      0.79        15



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


La precisión obtenida en dicho modelo es del 80%, siendo ese porcentaje de las prediciones realizadas de forma correcta.

Prueba del modelo:

In [30]:
def predict_mood_st(user_input):
    user_input_embedding = model.encode([user_input.lower()])
    mood_label = modelo_LR.predict(user_input_embedding)[0]
    return labels[mood_label][1]

frase_usuario = "Hoy me siento muy triste"
estado_animo = predict_mood_st(frase_usuario)
print(f"El estado de ánimo detectado es: {estado_animo}")

nuevas_frases = [
    "No puedo evitar sentir una gran tristeza hoy",  # Debería ser "Melancólico"
    "Siento que este es uno de los mejores días de mi vida",  # Debería ser "Alegre"
    "Hoy todo me da lo mismo, nada me interesa realmente",  # Debería ser "Ni fu ni fa"
    "No sé qué hacer, pero no estoy ni triste ni feliz",  # Debería ser "Ni fu ni fa"
    "Me siento increíblemente emocionado y lleno de energía",  # Debería ser "Alegre"
    "Todo parece gris y no veo nada que me anime",  # Debería ser "Melancólico"
    "Este día está siendo tan normal como cualquier otro",  # Debería ser "Ni fu ni fa"
    "No hay nada que me entusiasme, es como si estuviera vacío",  # Debería ser "Melancólico"
    "Estoy muy agradecido por todo, este día es fantástico",  # Debería ser "Alegre"
    "Hoy no me pasa nada en particular, no siento ni tristeza ni alegría"  # Debería ser "Ni fu ni fa"
]


for frase in nuevas_frases:
    print(f"La frase '{frase}' tiene el estado de ánimo: {predict_mood_st(frase)}")

El estado de ánimo detectado es: Melancólico
La frase 'No puedo evitar sentir una gran tristeza hoy' tiene el estado de ánimo: Melancólico
La frase 'Siento que este es uno de los mejores días de mi vida' tiene el estado de ánimo: Alegre
La frase 'Hoy todo me da lo mismo, nada me interesa realmente' tiene el estado de ánimo: Ni fu ni fa
La frase 'No sé qué hacer, pero no estoy ni triste ni feliz' tiene el estado de ánimo: Ni fu ni fa
La frase 'Me siento increíblemente emocionado y lleno de energía' tiene el estado de ánimo: Alegre
La frase 'Todo parece gris y no veo nada que me anime' tiene el estado de ánimo: Melancólico
La frase 'Este día está siendo tan normal como cualquier otro' tiene el estado de ánimo: Alegre
La frase 'No hay nada que me entusiasme, es como si estuviera vacío' tiene el estado de ánimo: Ni fu ni fa
La frase 'Estoy muy agradecido por todo, este día es fantástico' tiene el estado de ánimo: Alegre
La frase 'Hoy no me pasa nada en particular, no siento ni tristeza ni 

Como podemos ver, con SentenceTransformer, el modelo tiene mejores métricas.



Vamos a buscar extraer entidades nombradas de textos en español y luego usamos el modelo SpaCy para identificar y clasificar estas entidades dentro de una frase dada.

In [31]:
# Descargar stopwords en español
nltk.download('stopwords')

# Obtener las stopwords en español
spanish_stop_words = stopwords.words('spanish')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
nlp = spacy.load('es_core_news_md')

def extraer_entidades(texto):
    doc = nlp(texto)
    entidades = [(ent.text, ent.label_) for ent in doc.ents]
    return entidades

# Ejemplo de uso de la función de extracción de entidades
frase_usuario = "Quiero ver una historia de amor en París con personajes como Napoleón"
entidades = extraer_entidades(frase_usuario)
print("Entidades nombradas extraídas:", entidades)


Entidades nombradas extraídas: [('París', 'LOC'), ('Napoleón', 'PER')]


Definimos una función para que los usuarios ingresen sus preferencias o intereses sobre un tema específico.

In [33]:
# Función para ingresar preferencias del usuario
def ingresar_preferencias():
    preferencia_usuario = input("Describe la temática que te gustaría explorar: ")
    return preferencia_usuario

In [34]:
#Imprimimos información sobre los juegos de mesas
print(f"Cantidad de juegos de mesa: {len(board_games_df)}")
print(f"Primeras filas de juegos de mesa: {board_games_df.head()}")

Cantidad de juegos de mesa: 1000
Primeras filas de juegos de mesa:    rank                          game_name  \
0     1                  Brass: Birmingham   
1     2          Pandemic Legacy: Season 1   
2     3                         Gloomhaven   
3     4                           Ark Nova   
4     5  Twilight Imperium: Fourth Edition   

                                           game_href  geek_rating  avg_rating  \
0  https://boardgamegeek.com/boardgame/224517/bra...        8.415        8.60   
1  https://boardgamegeek.com/boardgame/161936/pan...        8.377        8.53   
2  https://boardgamegeek.com/boardgame/174430/glo...        8.349        8.59   
3  https://boardgamegeek.com/boardgame/342942/ark...        8.335        8.54   
4  https://boardgamegeek.com/boardgame/233078/twi...        8.240        8.60   

   num_voters                                        description  \
0     46836.0  Brass: Birmingham is an economic strategy game...   
1     53807.0  Pandemic Legacy is

In [35]:
# Procesar descripciones y títulos
board_game_desc = board_games_df['description'].fillna('')
movie_desc = movies_df['Description'].fillna('')
book_titles = books_df['Title'].fillna('')
all_descriptions = pd.concat([board_game_desc, movie_desc, book_titles], ignore_index=True)

# Vectorización
vectorizer = TfidfVectorizer(stop_words=spanish_stop_words, ngram_range=(1, 2))
desc_vectors = vectorizer.fit_transform(all_descriptions)

# Funciones para cada tipo de recomendación
def recomendar_juegos(similarity_scores):
    top_indices = similarity_scores[:len(board_game_desc)].argsort()[-3:][::-1]
    return [f"Juego de mesa: {board_games_df.iloc[idx]['game_name']}" for idx in top_indices]

def recomendar_peliculas(similarity_scores):
    start, end = len(board_game_desc), len(board_game_desc) + len(movie_desc)
    top_indices = similarity_scores[start:end].argsort()[-3:][::-1]
    return [
        f"Película: {movies_df.iloc[idx]['Title']}, Actores: {movies_df.iloc[idx]['Actors']}"
        for idx in top_indices
    ]

def recomendar_libros(similarity_scores):
    start = len(board_game_desc) + len(movie_desc)
    top_indices = similarity_scores[start:].argsort()[-3:][::-1]
    return [
        f"Libro: {books_df.iloc[idx]['Title']}, Autor: {books_df.iloc[idx]['Author']}"
        for idx in top_indices
    ]

# Función principal de recomendación
def recomendar_con_entidades(user_input):
    entidades = extraer_entidades(user_input)
    if entidades:
        print(f"Entidades encontradas en la frase: {entidades}")

    # Calcular la similitud del input con los vectores de descripción
    user_input_vector = vectorizer.transform([user_input])
    similarity_scores = cosine_similarity(user_input_vector, desc_vectors).flatten()

    recomendaciones = []

    # 3 recomendaciones para cada categoría
    recomendaciones += recomendar_juegos(similarity_scores)
    recomendaciones += recomendar_peliculas(similarity_scores)
    recomendaciones += recomendar_libros(similarity_scores)


    return recomendaciones[:9]

In [36]:
all_descriptions

,0
0,Brass: Birmingham is an economic strategy game...
1,Pandemic Legacy is a co-operative campaign gam...
2,Gloomhaven is a game of Euro-inspired tactica...
3,"In Ark Nova, you will plan and design a modern..."
4,Twilight Imperium (Fourth Edition) is a game o...
...,...
4825,A handy dictionary of mythology :
4826,The little book of life after death
4827,Orms
4828,Der


In [37]:
def mostrar_interfaz():

    estado_animo_input = widgets.Text(
        value='',
        placeholder='¿Cómo te sientes hoy?',
        description='Estado de Ánimo:',
        disabled=False
    )

    preferencia_input = widgets.Text(
        value='',
        placeholder='Describe la temática que te gustaría explorar',
        description='Preferencia:',
        disabled=False
    )

    boton_enviar = widgets.Button(description="Obtener Recomendaciones")


    output = widgets.Output()


    def on_button_clicked(b):
        with output:
            output.clear_output()

            estado_animo = estado_animo_input.value
            preferencia = preferencia_input.value


            estado_detectado = predict_mood_st(estado_animo)
            print(f"Estado de ánimo detectado: {estado_detectado}")


            recomendaciones = recomendar_con_entidades(preferencia)
            print("\nRecomendaciones:")
            for rec in recomendaciones:
                print(rec)


    boton_enviar.on_click(on_button_clicked)

    display(estado_animo_input, preferencia_input, boton_enviar, output)

# Ejecutar la interfaz
mostrar_interfaz()

Text(value='', description='Estado de Ánimo:', placeholder='¿Cómo te sientes hoy?')

Text(value='', description='Preferencia:', placeholder='Describe la temática que te gustaría explorar')

Button(description='Obtener Recomendaciones', style=ButtonStyle())

Output()

3 RECOMENDACIONES DE CADA CSV
